In [1]:
import tensorflow as tf
import sentencepiece as spm
from glob import glob
import os
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

In [2]:
# !wget https://f000.backblazeb2.com/file/malaya-model/bpe/sp10m.cased.t5.model
# !wget https://f000.backblazeb2.com/file/malaya-model/bpe/sp10m.cased.t5.vocab

In [3]:
# from tqdm import trange, tqdm

# vocab = 'sp10m.cased.t5.model'
# sp = spm.SentencePieceProcessor()
# sp.Load(vocab)

# vocab = {sp.id_to_piece(index): index for index in trange(sp.GetPieceSize())}
# merges = []
# for piece_l in tqdm(vocab.keys(), total=sp.GetPieceSize()):
#     for piece_r in vocab.keys():
#         merge = f"{piece_l}{piece_r}"
#         piece_id = vocab.get(merge, None)
#         if piece_id:
#             merges += [(piece_l, piece_r, piece_id)]
# merges = sorted(merges, key=lambda val: val[2])
# merges = [(val[0], val[1]) for val in merges]

In [4]:
# from json import dump
# from os import linesep, remove

# with open('out.vocab', 'w') as vocab_f:
#     with open('out.merged', 'w') as merges_f:
#         dump(vocab, vocab_f)
#         merges_f.writelines(map(lambda x: f"{x[0]} {x[1]}{linesep}", merges))

In [5]:
# from tokenizers import SentencePieceBPETokenizer

# tokenizer = SentencePieceBPETokenizer('out.vocab', 'out.merged')

In [6]:
vocab = '../sp10m.cased.ms-en.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)


class Encoder:
    def __init__(self, sp):
        self.sp = sp
        self.vocab_size = sp.GetPieceSize() + 100

    def encode(self, s):
        return self.sp.EncodeAsIds(s)

    def decode(self, ids, strip_extraneous = False):
        return self.sp.DecodeIds(list(ids))
    
encoder = Encoder(sp)

In [7]:
from tqdm import tqdm

@registry.register_problem
class Seq2Seq(text_problems.Text2TextProblem):

    @property
    def approx_vocab_size(self):
        return 32100
    
    @property
    def is_generate_per_split(self):
        return False
    
    @property
    def dataset_splits(self):
        return [{
            "split": problem.DatasetSplit.TRAIN,
            "shards": 1000,
        },
        {
            "split": problem.DatasetSplit.EVAL,
            "shards": 6,
        }]
    
    def generate_samples(self, data_dir, tmp_dir, dataset_split):
        
        encoder = Encoder(sp)
        
        pairs = [['train/left.txt', 'train/right.txt']]
        
        for pair in pairs:
        
            left = open(pair[0])
            right = open(pair[1])

            while True:
                try:
                    l, r = next(left), next(right)
                    l, r = l.strip(), r.strip()
                    l = encoder.encode(l) + [1]
                    r = encoder.encode(r) + [1]
                    yield {
                        'inputs': l,
                        'targets': r
                    }

                except StopIteration:
                    print('break')
                    break
            
            left.close()
            right.close()
                    
    def generate_encoded_samples(self, data_dir, tmp_dir, dataset_split):
        
        generator = self.generate_samples(data_dir, tmp_dir, dataset_split)
        for sample in tqdm(generator):
            yield sample

In [8]:
import os
import tensorflow as tf

os.system('rm -rf t2t-knowledge-graph/data')
DATA_DIR = os.path.expanduser('t2t-knowledge-graph/data')
TMP_DIR = os.path.expanduser('t2t-knowledge-graph/tmp')

In [9]:
tf.gfile.MakeDirs(DATA_DIR)
tf.gfile.MakeDirs(TMP_DIR)

In [10]:
from tensor2tensor.utils import registry
from tensor2tensor import problems

PROBLEM = 'seq2_seq'
t2t_problem = problems.problem(PROBLEM)
t2t_problem.generate_data(DATA_DIR, TMP_DIR)

0it [00:00, ?it/s]

INFO:tensorflow:Generating case 0.


INFO:tensorflow:Generating case 0.
99851it [00:26, 4109.06it/s]

INFO:tensorflow:Generating case 100000.


INFO:tensorflow:Generating case 100000.
199916it [00:53, 3826.44it/s]

...

INFO:tensorflow:Shuffling data...


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


INFO:tensorflow:Data shuffled.


INFO:tensorflow:Data shuffled.
